In [24]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.snowball import SnowballStemmer #porterstemmer?
from scipy.sparse.linalg import svds
from scipy.stats import percentileofscore
from surprise import SVD
from surprise import Dataset
from surprise import Reader

In [25]:
def getpredictions( predictions, userId ):
    user_pred = []
    for uid, iid, true_r, est, _ in predictions:
        if uid == userId :
            user_pred.append( (iid, est) )
    df_userp = pd.DataFrame( user_pred, columns = [ 'movieId', 'rating_pred'] )
    df_userp['movieId'] = df_userp['movieId'].astype(int)
    return( df_userp )

In [26]:
df_md_small = pd.read_csv('../movielens_small/metadata_clean.csv')
df_small = pd.read_csv('../movielens_small/ratings_clean.csv')

In [27]:
reader = Reader()
rating_data = Dataset.load_from_df(df_small[['userId', 'movieId', 'rating']], reader)
#rating_data.split(n_folds=5)

In [28]:
algo = SVD( n_factors = 100, n_epochs = 25, random_state=42 )
trainset = rating_data.build_full_trainset()
algo.fit(trainset)

In [29]:
# predict ratings for all pairs (u, i) that are not in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [30]:
df_md_small['genres'] = df_md_small['genres'].apply(ast.literal_eval)
df_md_small['cast'] = df_md_small['cast'].apply(ast.literal_eval)
df_md_small['keywords'] = df_md_small['keywords'].apply(ast.literal_eval)

In [31]:
df_md_small['director_'] = ( df_md_small['director'].
                            astype(str).
                            apply( lambda x: x.lower().replace( " ", "") ).
                            apply( lambda x: [x, x]) ) # more weights

In [32]:
df_md_small['cast_'] = ( df_md_small['cast'].
                        apply( lambda x: [i.lower().replace( " ", "") for i in x ]) )

In [33]:
df_md_small['keywords_'] = ( df_md_small['keywords'].
                            apply( lambda x: [SnowballStemmer( 'english' ).stem( i ) for i in x]).
                            apply( lambda x: [i.lower().replace( " ", "") for i in x ]) )

In [34]:
df_md_small['content'] = ( df_md_small['cast_'] +
                          df_md_small['director_'] +
                          df_md_small['keywords_'] +
                          df_md_small['genres'] ).apply( lambda x: ' '.join( x ) ) # combine all content features

In [35]:
content_matrix = CountVectorizer(
    analyzer = 'word',
    ngram_range = (1, 2),
    min_df = 0,
    stop_words = 'english').fit_transform(df_md_small['content'])
cosine_sim = cosine_similarity(content_matrix, content_matrix)

In [36]:
user_id = 50
movie_title = 'Terminator 2: Judgment Day'
df_md_small.loc[ df_md_small['title'] == movie_title, ['title', 'director', 'genres', 'cast'] ]

,title,director,genres,cast
522,Terminator 2: Judgment Day,James Cameron,"[Action, Thriller, Science Fiction]","[Arnold Schwarzenegger, Linda Hamilton, Robert..."


In [37]:
df_user_rating = ( df_small.loc[ df_small['userId'] == user_id ].
               merge(df_md_small, on = 'movieId').
               reset_index(drop = True) )

In [38]:
# user's top rated movies
df_user_rating[ ['movieId', 'title', 'rating', 'genres'] ].sort_values('rating', ascending = False ).head(10)

,movieId,title,rating,genres
41,589,Terminator 2: Judgment Day,5.0,"[Action, Thriller, Science Fiction]"
0,10,GoldenEye,4.0,"[Adventure, Action, Thriller]"
15,292,Outbreak,4.0,"[Action, Drama, Science Fiction, Thriller]"
44,780,Independence Day,4.0,"[Action, Adventure, Science Fiction]"
37,527,Schindler's List,4.0,"[Drama, History, War]"
35,480,Jurassic Park,4.0,"[Adventure, Science Fiction]"
31,440,Dave,4.0,[Comedy]
26,368,Maverick,4.0,"[Action, Adventure, Comedy, Drama, Western]"
25,367,The Mask,4.0,"[Romance, Comedy, Crime, Fantasy]"
24,357,Four Weddings and a Funeral,4.0,"[Comedy, Drama, Romance]"


In [39]:
df_userp = getpredictions( predictions, user_id )
df_userp.shape

(8979, 2)

In [40]:
# top recommendations using CF
df_userp.sort_values( 'rating_pred', ascending = False ).merge( df_md_small[['movieId', 'title']], on = 'movieId').head(10)

,movieId,rating_pred,title
0,56782,4.325372,There Will Be Blood
1,318,4.318308,The Shawshank Redemption
2,308,4.298549,Three Colors: White
3,608,4.286891,Fargo
4,2858,4.278215,American Beauty
5,2329,4.270856,American History X
6,1197,4.264183,The Princess Bride
7,3462,4.251772,Modern Times
8,2917,4.198536,Body Heat
9,926,4.197110,All About Eve


In [41]:
idx = np.asscalar( df_md_small.loc[ df_md_small['title'] == movie_title ].index.values[0] )
df_cbp = df_md_small[['movieId', 'title', 'genres']].reset_index( drop = True )
df_cbp[ 'sim' ] = cosine_sim[ idx ].flatten().tolist()
df_cbp.shape

(9082, 4)

In [42]:
# top recommendations using content-based filtering (CBF)
df_cbp.sort_values( 'sim', ascending = False ).head(11)

,movieId,title,genres,sim
522,589,Terminator 2: Judgment Day,"[Action, Thriller, Science Fiction]",1.000000
990,1240,The Terminator,"[Action, Thriller, Science Fiction]",0.367653
7404,73321,The Book of Eli,"[Action, Thriller, Science Fiction]",0.284029
3671,4691,Def-Con 4,"[Horror, Science Fiction, Thriller, Action]",0.243757
6821,56174,I Am Legend,"[Drama, Horror, Action, Thriller, Science Fict...",0.240048
953,1200,Aliens,"[Horror, Action, Thriller, Science Fiction]",0.235294
2024,2531,Battle for the Planet of the Apes,"[Action, Science Fiction]",0.234023
901,1127,The Abyss,"[Adventure, Action, Thriller, Science Fiction]",0.230812
2430,3032,The Omega Man,"[Action, Science Fiction, Thriller]",0.230205
2951,3702,Mad Max,"[Adventure, Action, Thriller, Science Fiction]",0.222566


In [43]:
df_c = df_userp.merge( df_cbp, on = 'movieId' )
x = df_c['rating_pred'].values
y = df_c['sim'].values
x_p = np.array( [percentileofscore(x, i, 'mean') for i in x] )
y_p = np.array( [percentileofscore(y, i, 'mean') for i in y] )
df_c['rating_p'] = x_p.tolist()
df_c['sim_p'] = y_p.tolist()

In [44]:
weight_cb = 0.5
score = x_p * ( 1 - weight_cb ) + y_p * weight_cb
df_c['score'] = score.tolist()

In [45]:
# top recommendations from hybrid recommendation engine
df_c.sort_values( 'score', ascending = False ).iloc[0:10][ ['title'] ].merge( df_md_small[['title', 'director', 'genres', 'cast']], on = 'title').reset_index( drop = True )

,title,director,genres,cast
0,Delicatessen,Jean-Pierre Jeunet,"[Comedy, Science Fiction, Fantasy]","[Dominique Pinon, Marie-Laure Dougnac, Jean-Cl..."
1,Mad Max 2: The Road Warrior,George Miller,"[Adventure, Action, Thriller, Science Fiction]","[Mel Gibson, Bruce Spence, Michael Preston, Ma..."
2,Ghost in the Shell,Mamoru Oshii,"[Action, Animation, Science Fiction]","[Atsuko Tanaka, Akio Ohtsuka, Iemasa Kayumi, K..."
3,Inception,Christopher Nolan,"[Action, Thriller, Science Fiction, Mystery, A...","[Leonardo DiCaprio, Joseph Gordon-Levitt, Elle..."
4,The Fifth Element,Luc Besson,"[Adventure, Fantasy, Action, Thriller, Science...","[Bruce Willis, Gary Oldman, Ian Holm, Milla Jo..."
5,The Matrix,Lana Wachowski,"[Action, Science Fiction]","[Keanu Reeves, Laurence Fishburne, Carrie-Anne..."
6,Mad Max,George Miller,"[Adventure, Action, Thriller, Science Fiction]","[Mel Gibson, Joanne Samuel, Hugh Keays-Byrne, ..."
7,Brazil,Terry Gilliam,"[Comedy, Science Fiction]","[Jonathan Pryce, Robert De Niro, Katherine Hel..."
8,The Terminator,James Cameron,"[Action, Thriller, Science Fiction]","[Arnold Schwarzenegger, Michael Biehn, Linda H..."
9,Return of the Jedi,Richard Marquand,"[Adventure, Action, Science Fiction]","[Mark Hamill, Harrison Ford, Carrie Fisher, Bi..."


In [46]:
# explanation for top recommendations from hybrid recommendation engine
df_c.sort_values( 'score', ascending = False ).iloc[0:10][['title', 'rating_pred', 'sim', 'rating_p', 'sim_p', 'score']].reset_index( drop = True )

,title,rating_pred,sim,rating_p,sim_p,score
0,Delicatessen,4.037706,0.121578,99.393028,98.496492,98.944760
1,Mad Max 2: The Road Warrior,3.832259,0.197446,96.653302,99.849649,98.251476
2,Ghost in the Shell,3.855393,0.125245,97.243568,98.624568,97.934068
3,Inception,3.825402,0.146119,96.452834,99.181423,97.817129
4,The Fifth Element,3.832258,0.126035,96.642165,98.752645,97.697405
5,The Matrix,4.014915,0.094407,99.292794,95.912685,97.602740
6,Mad Max,3.752800,0.222566,94.169729,99.905335,97.037532
7,Brazil,3.912941,0.092082,98.145673,95.667669,96.906671
8,The Terminator,3.738809,0.367653,93.690834,99.994431,96.842633
9,Return of the Jedi,3.948133,0.083497,98.680254,94.136318,96.408286
